# CH.10_Data_Aggregation_and_Group_Operations

In [1]:
import pandas as pd
import numpy as np

## 1. GroupBy Mechanics

In [3]:
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                  'key2' : ['one', 'two', 'one', 'two', 'one'],
                  'data1' : np.random.randn(5),
                  'data2' : np.random.randn(5)})

df

,key1,key2,data1,data2
0,a,one,0.551469,-0.998484
1,a,two,-0.760692,-0.618888
2,b,one,1.285910,1.868016
3,b,two,-1.798426,-2.925418
4,a,one,1.466995,0.899263


In [6]:
grouped = df['data1'].groupby(df['key1'])
grouped.head()

0    0.551469
1   -0.760692
2    1.285910
3   -1.798426
4    1.466995
Name: data1, dtype: float64

In [7]:
grouped.mean()

key1
a    0.419257
b   -0.256258
Name: data1, dtype: float64

In [9]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

key1  key2
a     one     1.009232
      two    -0.760692
b     one     1.285910
      two    -1.798426
Name: data1, dtype: float64

In [10]:
means.unstack()

key2,one,two
key1,,
a,1.009232,-0.760692
b,1.285910,-1.798426


In [11]:
states = np.array(['Ohio', 'Cali', 'Cali', 'Ohio', 'Ohio'])
years = np.array([2005, 205, 2006, 2005, 2006])
df['data1'].groupby([states, years]).mean()

Cali  205    -0.760692
      2006    1.285910
Ohio  2005   -0.623478
      2006    1.466995
Name: data1, dtype: float64

In [12]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,0.419257,-0.239370
b,-0.256258,-0.528701


In [13]:
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one   1.009232 -0.049610
     two  -0.760692 -0.618888
b    one   1.285910  1.868016
     two  -1.798426 -2.925418

In [14]:
df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

In [16]:
for name, group in df.groupby('key1') :
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one  0.551469 -0.998484
1    a  two -0.760692 -0.618888
4    a  one  1.466995  0.899263
b
  key1 key2     data1     data2
2    b  one  1.285910  1.868016
3    b  two -1.798426 -2.925418


In [17]:
for (k1, k2), group in df.groupby(['key1', 'key2']) :
    print((k1, k2))
    print(group)

('a', 'one')
  key1 key2     data1     data2
0    a  one  0.551469 -0.998484
4    a  one  1.466995  0.899263
('a', 'two')
  key1 key2     data1     data2
1    a  two -0.760692 -0.618888
('b', 'one')
  key1 key2    data1     data2
2    b  one  1.28591  1.868016
('b', 'two')
  key1 key2     data1     data2
3    b  two -1.798426 -2.925418


In [19]:
pieces = dict(list(df.groupby('key1')))
pieces['b']

,key1,key2,data1,data2
2,b,one,1.285910,1.868016
3,b,two,-1.798426,-2.925418


In [22]:
df.dtypes

key1      object
key2      object
data1    float64
data2    float64
dtype: object

In [24]:
grouped = df.groupby(df.dtypes, axis = 1)
for dtype, group in grouped :
    print(dtype)
    print(group)

float64
      data1     data2
0  0.551469 -0.998484
1 -0.760692 -0.618888
2  1.285910  1.868016
3 -1.798426 -2.925418
4  1.466995  0.899263
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


In [25]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],
                                    [1,3,5,1,3]],
                                    names = ['city', 'tensor'])

In [26]:
hier_df = pd.DataFrame(np.random.randn(4,5), columns = columns)
hier_df

city          US                            JP          
tensor         1         3         5         1         3
0      -1.234270 -0.165546  1.669804  1.435218 -0.495893
1      -0.903382  1.691233  1.392761 -0.606852  1.219343
2       1.293730 -0.401898  0.766791  0.707717 -1.827681
3       0.949330 -0.530591  0.874663  1.107031 -0.444467

In [27]:
hier_df.groupby(level = 'city', axis = 1).count()

city,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


In [28]:
grouped = df.groupby('key1')
grouped['data1'].quantile(0.9)

key1
a    1.283890
b    0.977477
Name: data1, dtype: float64

In [29]:
def peak_to_peak(arr) :
    return arr.max() - arr.min()

grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,2.227687,1.897747
b,3.084336,4.793434


In [30]:
grouped.describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0  0.419257  1.119713 -0.760692 -0.104612  0.551469  1.009232   
b      2.0 -0.256258  2.180955 -1.798426 -1.027342 -0.256258  0.514826   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     1.466995   3.0 -0.239370  1.004185 -0.998484 -0.808686 -0.618888   
b     1.285910   2.0 -0.528701  3.389470 -2.925418 -1.727060 -0.528701   

                          
           75%       max  
key1                      
a     0.140188  0.899263  
b     0.669657  1.868016

## 2. Data Aggregation

In [52]:
frame = pd.DataFrame({'data1' : np.random.randn(1000),
                     'data2' : np.random.randn(1000)})

In [53]:
quartiles = pd.cut(frame.data1, 4)
quartiles[:10]

0     (0.426, 2.146]
1    (-1.294, 0.426]
2     (0.426, 2.146]
3    (-1.294, 0.426]
4    (-1.294, 0.426]
5     (0.426, 2.146]
6    (-1.294, 0.426]
7     (0.426, 2.146]
8     (0.426, 2.146]
9    (-1.294, 0.426]
Name: data1, dtype: category
Categories (4, interval[float64]): [(-3.022, -1.294] < (-1.294, 0.426] < (0.426, 2.146] < (2.146, 3.867]]

In [54]:
def get_stats(group) :
    return {'min' : group.min(), 'max' : group.max(),
           'count' : group.count(), 'mean' : group.mean()}

In [58]:
grouped = frame.data2.groupby(quartiles)
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
"(-3.022, -1.294]",-2.247905,3.202250,80.0,0.059563
"(-1.294, 0.426]",-3.722824,3.996671,559.0,0.049780
"(0.426, 2.146]",-2.118520,2.500116,347.0,0.047112
"(2.146, 3.867]",-1.288215,1.094758,14.0,0.145541


In [60]:
grouping = pd.qcut(frame.data1, 10, labels = False)
grouped = frame.data2.groupby(grouping)
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
0,-3.722824,3.202250,100.0,0.026712
1,-2.102581,3.996671,100.0,0.156176
2,-2.127293,2.434292,100.0,-0.010677
3,-2.145675,2.059312,100.0,0.077744
4,-2.005761,1.776393,100.0,0.038612
5,-2.180200,2.578189,100.0,-0.014905
6,-2.304759,2.164803,100.0,0.172239
7,-2.118520,1.802799,100.0,-0.095276
8,-2.044234,1.828767,100.0,0.092362


In [61]:
s = pd.Series(np.random.randn(6))
s[::2] = np.nan
s

0         NaN
1   -0.061253
2         NaN
3    0.196953
4         NaN
5   -0.658053
dtype: float64

## 3. General split-apply-combine

In [62]:
states = ['Ohio', 'New', 'Ver', 'Flor', 'Ore', 'Nev', 'Cali', 'Ida']
group_key = ['East']*4 + ['West']*4
data = pd.Series(np.random.randn(8), index = states)
data

Ohio   -0.498793
New     0.227878
Ver    -0.331391
Flor   -1.648169
Ore     0.545557
Nev    -1.473525
Cali   -1.801170
Ida     2.034741
dtype: float64

In [63]:
data[['Ver', 'Nev', 'Ida']] = np.nan
data

Ohio   -0.498793
New     0.227878
Ver          NaN
Flor   -1.648169
Ore     0.545557
Nev          NaN
Cali   -1.801170
Ida          NaN
dtype: float64

In [64]:
data.groupby(group_key).mean()

East   -0.639695
West   -0.627807
dtype: float64

In [67]:
fill_mean = lambda g : g.fillna(g.mean())
data.groupby(group_key).apply(fill_mean)

Ohio   -0.498793
New     0.227878
Ver    -0.639695
Flor   -1.648169
Ore     0.545557
Nev    -0.627807
Cali   -1.801170
Ida    -0.627807
dtype: float64

In [68]:
fill_values = {'East' : 0.5, 'West' : -1}
fill_func = lambda g: g.fillna(fill_values[g.name])
data.groupby(group_key).apply(fill_func)

Ohio   -0.498793
New     0.227878
Ver     0.500000
Flor   -1.648169
Ore     0.545557
Nev    -1.000000
Cali   -1.801170
Ida    -1.000000
dtype: float64

In [56]:
a = pd.DataFrame(np.arange(0,21))[]
a.quantile(0.9)

0    18.0
Name: 0.9, dtype: float64

In [50]:
np.arange(0,11)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10])

## 4. Pivot Tables and Cross-Tabulation

In [72]:
import pandas_datareader.data as web

all_data = {ticker : web.get_data_yahoo(ticker)
           for ticker in ['AAPL', 'IBM', 'MSFT', 'GOOG']}
price = pd.DataFrame({ticker : data['Adj Close']
                     for ticker, data in all_data.items()})
volume = pd.DataFrame({ticker : data['Volume']
                      for ticker, data in all_data.items()})